# Vexing Vexillology

This is a late entry for self-gratification of [the Spelling Bee riddle](https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/). In short, which Spelling Bee board gives us the highest score?

A valid board is a set of 7 unique letters from which the player must form English words. Valid words must contain only letters from the board, must include the central letter, must be at least 4 letters long, and may contain a letter more than once. Boards must not include "S", and there must be at least one **pangram** (a word using every letter on the board). As for scoring:

> Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, seven-letter words are worth 7 points, etc. Words that use all of the seven letters in the honeycomb are known as “pangrams” and earn 7 bonus points (in addition to the points for the length of the word).

The prompt was scored against [this dictionary](https://norvig.com/ngrams/enable1.txt).

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
from scipy import special

num_possible_boards = special.binom(25, 6) * special.binom(19, 1)
print(num_possible_boards)

3364900.0


My gut went right to a Trie, but we don't care about the letters per word, but the unique letters per word. The complicating factor is the required central letter.

Now by brute force, there are

$$\binom{25}{6}\times\binom{19}{1}\approx3.7\times10^{6}\,\mathrm{boards}$$


If you give one second to check each board, we're looking at just over a month for a solution. But here's a key insight. Since valid boards must contain one pangram, we only need to concern ourselves with the possible boards derived from 7+ length words in our dictionary. Put another way: if we had a board from which no word in our dictionary could be formed using all 7 letters at least once, then the board is invalid.

At the same time, we can create a Trie-adjacent structure that maps a set of unique letters derived from our dictionary words to that set's overall score. For example,  We can use this while checking a board's score.

In [3]:
def get_words():
    res = urllib.request.urlopen('https://norvig.com/ngrams/enable1.txt')
    return res.read().decode('utf-8').upper().split('\n')


In [4]:
import urllib

# Keeping with the spirit of the problem, a hive is the
# set of distinct (lexicographically ordered) letters of a word
def make_hive(word):
    return ''.join(sorted(set(word)))

def score(word):
    bonus = (7 if len(set(word)) == 7 else 0)
    base = (1 if len(word) == 4 else len(word))
    return bonus + base

def get_scores(words):
    scores = {}
    for cur_word in words:
        cur_hive = make_hive(cur_word)
        # With > 7 distinct letters, a word can't be derived from a board
        # Words < 4 letters aren't scored, so ignore them too
        if len(cur_hive) <= 7 \
                and len(cur_word) >= 4 \
                and 'S' not in cur_word:
            scores[cur_hive] = scores.get(cur_hive, 0) + score(cur_word)
    return scores

scores = get_scores(get_words())

Check our work quickly.

In [5]:
assert( max(list(map(lambda x: len(x), scores))) == 7)

Now we extract the seven letter hives from our score map, and generate the possible boards by noting that for each hive, there are 7 potential boards, one for each letter as the "central letter" or "mandatory letter".

In [6]:
candidates = list(filter(lambda x: len(x) == 7, scores))
boards = [(c,letter) for c in candidates for letter in c]

We're close. Now consider a single board. We want to check every possible subset of the board against our scores to see how many points can be scored from that subset. Do that for each board and we're good to go!

In [7]:
from itertools import combinations

def all_board_subsets(board):
    (letters, middle) = board
    subsets = []
    for i in range(1, 8):
        for c in combinations(letters, i):
            if middle in c:
                subsets.append(''.join(sorted(c)))
    return subsets

def get_board_score(board, scores):
    return sum([scores[s] for s in all_board_subsets(board) if s in scores])

def best_board(boards, scores):
    return max((get_board_score(b, scores), b) for b in boards)


%time best_board(boards, scores)

CPU times: user 9.13 s, sys: 0 ns, total: 9.13 s
Wall time: 9.27 s


(3898, ('AEGINRT', 'R'))

There you have it, just shy of 4000 points, a board with "R" in the center. Now if the Times wanted to put the "lol" in vexillology they would publish this board as a prank and let their readers go apopleptic.

This puzzle begs for a board solver, i.e. given a board, which words can be formed? A trie again seems useful.

In [8]:
from util.trie import Trie
riddle_dict = Trie(get_words())

Alas since there's no upper bound on word length, we can't just permute all the possible hives on a board without a massive time penalty. Once again, parsing the dictionary itself is the key to success.

Here's a board from today. We need a map from hives to the possible words. Then we generate all possible hives from the board, look them up in our map, and optionally get a score if we like.

In [9]:
def hive_map(words):
    final_map = {}
    for w in words:
        if len(w) < 4 or 'S' in w:
            continue
        cur_hive = make_hive(w)
        if cur_hive not in final_map:
            final_map[cur_hive] = []
        final_map[cur_hive].append(w)
    return final_map

hives_to_words = hive_map(get_words())

board_20200125 = ('MWHAOND', 'A')

def get_all_words(board, word_map):
    all_words = []
    for s in all_board_subsets(board):
        cur_words = word_map.get(s, None)
        if cur_words:
            all_words += cur_words
    return sorted(all_words)

get_all_words(board_20200125, hives_to_words)

['ADMAN',
 'ADOWN',
 'AMAH',
 'AMMO',
 'AMMONO',
 'ANNA',
 'ANOA',
 'ANON',
 'DADA',
 'DADO',
 'DAHOON',
 'DAMAN',
 'DAMN',
 'DAWN',
 'DONA',
 'DONNA',
 'DOODAD',
 'HAHA',
 'HAMADA',
 'HAMMADA',
 'HAND',
 'HODAD',
 'HONAN',
 'HONDA',
 'HOWDAH',
 'HWAN',
 'MADAM',
 'MADMAN',
 'MADONNA',
 'MADWOMAN',
 'MAMA',
 'MAMMA',
 'MAMMON',
 'MANA',
 'MANANA',
 'MANHOOD',
 'MANNA',
 'MANNAN',
 'MANO',
 'MAWN',
 'MOAN',
 'MOMMA',
 'MONAD',
 'NAAN',
 'NADA',
 'NANA',
 'NOMA',
 'NOMAD',
 'NONA',
 'NONMAN',
 'WAHOO',
 'WAND',
 'WHAM',
 'WHAMMO',
 'WHAMO',
 'WHOA',
 'WOAD',
 'WOMAN',
 'WOMANHOOD',
 'WOODMAN']